In [2]:
import torch
from transformers import T5ForConditionalGeneration, T5Tokenizer, Trainer, TrainingArguments
from DeepNLGDatasetReader import run_parser
from Utils import deepnlg_entry_to_examples

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

### Train planner on DeepNLG data

In [3]:
train_entries = run_parser('deepnlg/train')
dev_entries = run_parser('deepnlg/dev')

In [28]:
train_examples = []
dev_examples = []

for entry in train_entries:
    train_examples.extend(deepnlg_entry_to_examples(entry))

for entry in dev_entries:
    dev_examples.extend(deepnlg_entry_to_examples(entry))

2268